In [6]:
import pickle
import pandas as pd

In [ ]:
data = pd.read_csv("E:\\NLP_Recommender_System\\merge_comments\\video_comments.csv")

In [8]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4492 entries, 0 to 4491
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   video_id      4492 non-null   object
 1   all_comments  4492 non-null   object
dtypes: object(2)
memory usage: 70.3+ KB


In [9]:
data.shape

(4492, 2)

In [10]:
data.head(3)

,video_id,all_comments
0,YgAuFqEs6yk,Amazing_ This is true. Sometimes things happen...
1,bNmRr-BYnxA,Bravo!_ A successful mindset isn't something t...
2,FVUkKKc3Vvk,"I enjoyed😊 her book, My Side of the River._ So..."


In [ ]:
first_video = data.head(1)
first_video

,video_id,all_comments
0,YgAuFqEs6yk,Amazing_ This is true. Sometimes things happen...


In [12]:
first_video["comment_list"] = first_video["all_comments"].apply(lambda x: [c.strip() for c in x.split('_') if c.strip()])

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_26208\1198387123.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  first_video["comment_list"] = first_video["all_comments"].apply(lambda x: [c.strip() for c in x.split('_') if c.strip()])


In [13]:
first_video

,video_id,all_comments,comment_list
0,YgAuFqEs6yk,Amazing_ This is true. Sometimes things happen...,"[Amazing, This is true. Sometimes things happe..."


In [14]:
df_exploded = first_video[["video_id", "comment_list"]].explode("comment_list").rename(columns={"comment_list": "comment"})

In [15]:
df_exploded

,video_id,comment
0,YgAuFqEs6yk,Amazing
0,YgAuFqEs6yk,This is true. Sometimes things happen that are...
0,YgAuFqEs6yk,Powerful & Amazing
0,YgAuFqEs6yk,I can’t take a guy with dreads and a Detroit h...
0,YgAuFqEs6yk,How They Both Move Close To Each-other As They...
...,...,...
0,YgAuFqEs6yk,It's the way they're counted and validated by ...
0,YgAuFqEs6yk,Youtube has been around for decades and folk l...
0,YgAuFqEs6yk,The best
0,YgAuFqEs6yk,She's bomb. But black folks give entertainment...


In [16]:
import re

def clean_comment(text):
    text = re.sub(r"http\S+", "", text)                # URLs
    text = re.sub(r"@\w+", "", text)                   # Mentions
    text = re.sub(r"#[\w-]+", "", text)                # Hashtags
    text = re.sub(r"[^\w\s]", "", text)                # Punctuation & emojis
    text = re.sub(r"\s+", " ", text)                   # Extra spaces
    return text.strip().lower()

# Step 4: Clean the comments
df_exploded["clean_comment"] = df_exploded["comment"].apply(clean_comment)


In [17]:
df_exploded

,video_id,comment,clean_comment
0,YgAuFqEs6yk,Amazing,amazing
0,YgAuFqEs6yk,This is true. Sometimes things happen that are...,this is true sometimes things happen that aren...
0,YgAuFqEs6yk,Powerful & Amazing,powerful amazing
0,YgAuFqEs6yk,I can’t take a guy with dreads and a Detroit h...,i cant take a guy with dreads and a detroit ha...
0,YgAuFqEs6yk,How They Both Move Close To Each-other As They...,how they both move close to eachother as they ...
...,...,...,...
0,YgAuFqEs6yk,It's the way they're counted and validated by ...,its the way theyre counted and validated by th...
0,YgAuFqEs6yk,Youtube has been around for decades and folk l...,youtube has been around for decades and folk l...
0,YgAuFqEs6yk,The best,the best
0,YgAuFqEs6yk,She's bomb. But black folks give entertainment...,shes bomb but black folks give entertainment e...


In [1]:
!pip install torch

Defaulting to user installation because normal site-packages is not writeable


Why Use Dimensionality Reduction?
Your sentence embeddings (e.g., from MiniLM) are usually 384-dimensional.

Clustering methods like HDBSCAN and KMeans work better in lower dimensions—typically around 5 to 50.

Dimensionality reduction removes noise and improves cluster quality + speeds up computations.



In [19]:
from sentence_transformers import util, SentenceTransformer
# Pre-calculate embeddings
embedding_model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")
text = []
for x in df_exploded['clean_comment']:
    text.append(x)
embeddings = embedding_model.encode(text, show_progress_bar=True)

Batches:   0%|          | 0/4 [00:00<?, ?it/s]

In [79]:
text

['amazing',
 'this is true sometimes things happen that arent positive but out of every negative thing that happens there is a seed of opportunity for growth and change i am happy these people were able to put their differences aside and were able to do whats best for the child i am going through something similar however i dont want to be enemies with the mother of my children i will always love her and respect her for everything she gave me',
 'powerful amazing',
 'i cant take a guy with dreads and a detroit hat on on a ted talk seriously at all sorry',
 'how they both move close to eachother as they talk',
 'now these are 2 beautiful grown black adultsnot cardi b and offset lol',
 'thank you',
 'i think its so wonderful how they are speaking to an audience but really to each other so its genuine and it makes their message so relatable',
 'parents need to watch this',
 'two min in and im wishing all women would think this way 50 50 parenting support ill stop there or ill rant',
 'bea

In [20]:
embeddings

array([[-0.09810252, -0.00468978,  0.0017602 , ..., -0.00239507,
        -0.03424583,  0.09159009],
       [-0.06706116,  0.06538443,  0.00657029, ...,  0.00469585,
         0.07295034,  0.00661319],
       [-0.03640243,  0.05614042, -0.03111582, ...,  0.02659713,
         0.03404128,  0.04038696],
       ...,
       [-0.11371872,  0.02451536, -0.05797755, ...,  0.06054622,
         0.07351045,  0.13380738],
       [ 0.05567351, -0.03220436, -0.04690749, ..., -0.09492762,
        -0.02058854, -0.01266839],
       [ 0.02489519,  0.0207468 ,  0.04926012, ..., -0.04400522,
         0.04245618,  0.01658163]], dtype=float32)

→ Use UMAP for sentence embeddings and topic modeling. It's the default for BERTopic for a reason.
→ PCA is simpler and faster but doesn’t capture complex structures in language as well.



🔹 n_neighbors=15
Controls how local or global your reduction is.

Small = more local structure preserved (more detailed clustering).

Large = more global structure (smoother, broader clusters).

15 is a good default; go lower (5–10) for tiny datasets, higher (30–50) for large.

🔹 n_components=5
Final dimensionality after reduction (e.g., 5D space).

Typical range: 5–15 for BERTopic.

Too low → lose detail; too high → clustering becomes noisy.

🔹 min_dist=0.0
How tightly UMAP packs points in the reduced space.

0.0 = max compression (tight, distinct clusters).

0.5 = more spread out.

For clustering: lower is better.

🔹 metric='cosine'
Distance function between points.

Use "cosine" for text embeddings—it captures semantic similarity better than Euclidean.

🔹 random_state=42
For reproducibility. Set this so you get consistent results each time.

✅ Summary – Stick With UMAP
Unless you want to do a super-fast prototype or you’re working with numeric structured features, UMAP is the way to go for text embeddings like you're doing here.

In [62]:
from umap import UMAP

umap_model = UMAP(n_neighbors=10, n_components=2, min_dist=0.0, metric='cosine', random_state=42)

🤖 HDBSCAN — Quick Recap
What it is:
Hierarchical Density-Based Spatial Clustering of Applications with Noise

A density-based clustering algorithm.

Works well when:

You don’t know the number of clusters.

You expect uneven cluster sizes.

You’re fine with having some points marked as noise (-1).

🔧 Your Current HDBSCAN Setup
python
Copy
Edit
HDBSCAN(
    min_cluster_size=150,
    metric='euclidean',
    cluster_selection_method='eom',
    prediction_data=True
)
Let’s walk through these:

Hyperparameter	Meaning
min_cluster_size=150	Minimum number of points to form a cluster. High value = fewer, larger clusters. For small datasets, too high. Try 20–50.
metric='euclidean'	Works okay, but for sentence embeddings, use 'cosine'
cluster_selection_method='eom'	“Excess of Mass” method. Use "eom" (default) for well-separated clusters. "leaf" gives more fine-grained clusters.
prediction_data=True	Needed if you want to assign new points to existing clusters later. Set to True just in case.
🧠 Should You Use HDBSCAN?
✅ Use HDBSCAN if:
You don’t know the number of topics ahead of time.

You're working with UMAP-reduced embeddings.

You're okay with some comments being labeled as outliers (-1).

You want natural cluster shapes and densities (realistic for comments).

⚠️ When HDBSCAN Might Struggle:
Very small datasets (e.g., <100 points).

If you need every comment to belong to some cluster (because HDBSCAN can label points as noise).

🧩 Alternatives to HDBSCAN
Alternative	When to Use	Pros	Cons
KMeans	You want exact number of clusters	Fast, simple	Assumes equal-sized spherical clusters
Agglomerative Clustering	You want hierarchical topics	No “noise” points	Need to define n_clusters
Spectral Clustering	Complex data shapes, small-medium sets	Good on non-convex shapes	Memory-heavy on large sets
BERTopic's Class-based TF-IDF (c-TF-IDF)	You don't care about raw embeddings	Text-based only	Less flexible, not good for semantic embeddings


In [67]:
from hdbscan import HDBSCAN
hdbscan_model = HDBSCAN(
    min_cluster_size=4,         # Try 20–50 depending on comment count
    metric='euclidean',             # Better for text
    cluster_selection_method='eom',
    prediction_data=True
)


🧠 So what does the vectorizer_model do?
The vectorizer_model is used to create a bag-of-words representation of each topic, and this is what BERTopic uses to extract the "top words" for each topic cluster.

➕ In other words:
After you’ve clustered documents (using embeddings + HDBSCAN),

BERTopic uses all the documents within each topic

It vectorizes them with the CountVectorizer

🧪 Do you need CountVectorizer?
Yes, unless you use a representation model that completely overrides the keyword extraction (like OpenAI).

So in practice:

✅ Yes, you should still provide a vectorizer_model, because:

It helps initial keyword extraction

Many visualizations rely on it (e.g., heatmaps, bar charts)

It's fast, doesn’t interfere with embeddings

In [50]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer_model = CountVectorizer(
    stop_words="english",   # Removes basic stopwords
    min_df=1,                # Ignore words that appear in only 1 comment
    ngram_range=(1, 2)       # Allows bigrams like "great talk"
)


In [51]:

from bertopic.representation import KeyBERTInspired
# KeyBERT
keybert_model = KeyBERTInspired()

# Part-of-Speech
representation_model = {
    "KeyBERT": keybert_model,
}

In [80]:
from bertopic import BERTopic

topic_model = BERTopic(

  # Pipeline models
  embedding_model=embedding_model,
  umap_model=umap_model,
  hdbscan_model=hdbscan_model,
  vectorizer_model=vectorizer_model,
  representation_model=representation_model,

  # Hyperparameters
  top_n_words=3,
  verbose=True
)

topics, probs = topic_model.fit_transform(text, embeddings)

2025-04-09 00:35:52,935 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2025-04-09 00:35:53,288 - BERTopic - Dimensionality - Completed ✓
2025-04-09 00:35:53,299 - BERTopic - Cluster - Start clustering the reduced embeddings
2025-04-09 00:35:53,321 - BERTopic - Cluster - Completed ✓
2025-04-09 00:35:53,333 - BERTopic - Representation - Fine-tuning topics using representation models.
2025-04-09 00:35:56,253 - BERTopic - Representation - Completed ✓


In [81]:
keybert_topic_labels = {topic: " | ".join(list(zip(*values))[0][:3]) for topic, values in topic_model.topic_aspects_["KeyBERT"].items()}
topic_model.set_topic_labels(keybert_topic_labels)

In [82]:
topic_model.get_topic_info()

,Topic,Count,Name,CustomName,Representation,KeyBERT,Representative_Docs
0,-1,15,-1_powerful_talk_benefit,church thought | talk raising | parents sharing,"[powerful, talk, benefit]","[church thought, talk raising, parents sharing...",[church of thought thats a common misconceptio...
1,0,34,0_black_ted_like,love familys | gwendolyn williams | relatable ...,"[black, ted, like]","[love familys, gwendolyn williams, relatable p...",[black love on display amazing for the world t...
2,1,25,1_child_let_love,let child | parenting | let kid,"[child, let, love]","[let child, parenting, let kid, child people, ...",[my sons father and i had an on again off agai...
3,2,22,2_beautiful_thank_love,beautiful | absolutely beautiful | truly beaut...,"[beautiful, thank, love]","[beautiful, absolutely beautiful, truly beauti...","[beautiful, this is truly beautiful and a lot ..."
4,3,14,3_hi_awesome 9aco_minutes away,early hi | hi | early,"[hi, awesome 9aco, minutes away]","[early hi, hi, early, hi detroit, yeet hi, yes...","[hi, hi first, detroit 10 minutes away from 14..."
5,4,14,4_dont_theyre_yeah,youtube pedantry | listening video | video ita...,"[dont, theyre, yeah]","[youtube pedantry, listening video, video ital...",[because theyre doing this for clout 100 guara...


In [83]:
topic_model.visualize_topics(custom_labels=True)

C:\Users\Lenovo\AppData\Roaming\Python\Python312\site-packages\umap\umap_.py:1952: UserWarning:

n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.



In [72]:
topic_model.visualize_hierarchy(custom_labels=True)

Although visualizing the documents in 2-dimensional gives an idea of their underlying structure, there is a risk involved.

Visualizing the documents in 2-dimensional space means that we have lost significant information since the original embeddings were more than 384 dimensions. Condensing all that information in 2 dimensions is simply not possible. In other words, it is merely an approximation, albeit quite an accurate one.

In [85]:
topic_model.visualize_documents(text, custom_labels=True, hide_annotations=True)

In [75]:
embedding_model = "sentence-transformers/all-MiniLM-L6-v2"
topic_model.save("my_model_dir", serialization="safetensors", save_ctfidf=True, save_embedding_model=embedding_model)

In [2]:
from bertopic import BERTopic
from sentence_transformers import SentenceTransformer

# Define embedding model
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")

# Load model and add embedding model
loaded_model = BERTopic.load("my_model_dir", embedding_model=embedding_model)

In [86]:
loaded_model.visualize_heatmap()

In [3]:
loaded_model.visualize_barchart()